# Bit serial computation - Multiplication

The notebook illustrates how bit-serial multiplicaton of an operand `A` by an operand `B`, where the `A` operand is viewed at the bit level, can be treated as a tensor computation. This computation can be viewed as a tensor computation where the bit-level representation of `A` is achieved with a sparse fiber with a 1 at those coordinates that match the bit-positions with a 1 in the binary representation of the value. The operand `B` is simply represented as a scalar value. As a result this computation can be represented with the following Einsum:

$$
Z = A_j \times 2^j \times B
$$

## Setup

The first step is to set up the environment and create some tensors

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=movie

## Configure some tensors

In [ ]:
# Default value for the number of bits in the operand A
J = 8

tm = TensorMaker("dot product inputs")

tm.addTensor("A_J", rank_ids=["J"], shape=[J], density=0.5, interval=1, seed=0, color="blue")

tm.displayControls()

## Create and display the tensors

In [ ]:
A_J = tm.makeTensor("A_J")

#
# Calculate binary value of A from bit-wise represenation
#
a_value = 0
for j, _ in A_J:
    a_value += 2**j


B = Tensor(rank_ids=[], name="B", color="green")

b = B.getRoot()
b <<= 5

print(f"A_J (with value {a_value})")
displayTensor(A_J)

print("B")
displayTensor(B)

## Create power array

Although the original Einsum notation includes a multiplication by a value that is a function only of an index value (`2^j`), this code will express that as a multiplicaton by a value from a constant rank-1 tensor (`pow2`). In reality, this would probably be implemented directly in hardware (in this case as a **shift**).

In [ ]:
pow2 = Tensor(rank_ids=["J"], shape=[J], name="Pow2", color="lightblue")

pow2_j = pow2.getRoot()

for j, pow2_ref in pow2_j.iterShapeRef():
    pow2_ref <<= 2 ** j
    
displayTensor(pow2)

## Serial execution

Observations:

- Elapsed time is proportional to the occupancy of fiber in the `J` rank of `A_J`.

In [ ]:
z = Tensor(rank_ids=[], name="Product")

a_j = A_J.getRoot()
b_val = B.getRoot()
pow2_j = pow2.getRoot()

z_ref = z.getRoot()

canvas = createCanvas(A_J, B, pow2, z)

for j, (a_val, pow2_val) in a_j & pow2_j:
    z_ref += (a_val * b_val) * pow2_val
    canvas.addFrame((j,),(0,),(j,), (0,))
        
displayTensor(z)
displayCanvas(canvas)